# Algorithmic Fairness, Accountability, and Ethics, Spring 2025

## Mandatory Assignment 2

Please use the following code to prepare the dataset.
 

In [22]:
from folktables.acs import adult_filter
from folktables import ACSDataSource
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.optimize import fmin_tnc
import pandas as pd

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=False)

feature_names = ['AGEP', # Age
                 "CIT", # Citizenship status
                 'COW', # Class of worker
                 "ENG", # Ability to speak English
                 'SCHL', # Educational attainment
                 'MAR', # Marital status
                 "HINS1", # Insurance through a current or former employer or union
                 "HINS2", # Insurance purchased directly from an insurance company
                 "HINS4", # Medicaid
                 "RAC1P", # Recoded detailed race code
                 'SEX']

target_name = "PINCP" # Total person's income

def data_processing(data, features, target_name:str, threshold: float = 35000):
    df = data
    ### Adult Filter (STARTS) (from Foltktables)
    df = df[~df["SEX"].isnull()]
    df = df[~df["RAC1P"].isnull()]
    df = df[df['AGEP'] > 16]
    df = df[df['PINCP'] > 100]
    df = df[df['WKHP'] > 0]
    df = df[df['PWGTP'] >= 1]
    ### Adult Filter (ENDS)
    ### Groups of interest
    sex = df["SEX"].values
    race = df["RAC1P"].values
    ### Target
    df["target"] = df[target_name] > threshold
    target = df["target"].values
    df = df[features + ["target", target_name]] ##we want to keep df before one_hot encoding to make Bias Analysis
    df_processed = df[features].copy()
    cols = [ "HINS1", "HINS2", "HINS4", "CIT", "COW", "SCHL", "MAR", "SEX", "RAC1P"]
    df_processed = pd.get_dummies(df_processed, prefix=None, prefix_sep='_', dummy_na=False, columns=cols, drop_first=True)
    df_processed = pd.get_dummies(df_processed, prefix=None, prefix_sep='_', dummy_na=True, columns=["ENG"], drop_first=True)
    # Adding an intercept column, for fitting log reg
    df_processed['Intercept']=1
    # Reorder the columns so that 'x0' becomes the first column
    df_processed = df_processed[['Intercept'] + [col for col in df_processed.columns if col != 'Intercept']]

    return df_processed, df, target, sex, race

data, data_original, target, groupsex, grouprace = data_processing(acs_data, feature_names, target_name)

X_train, X_test, y_train, y_test, groupsex_train, groupsex_test, grouprace_train, grouprace_test = train_test_split(
    data, target, groupsex, grouprace, test_size=0.2, random_state=0)

X_train=X_train.astype(float)
y_train=y_train.astype(float)
X_test=X_test.astype(float)
y_test=y_test.astype(float)
X_train = X_train.to_numpy()

## Logistic regression model without any fairness

In [23]:
# Sigmoid function
def sigmoid(beta, X):
    return 1/(1+np.exp(-(X @ beta)))

# Logistic loss 
def logistic_loss(beta, X, y, lambda_, gamma_):
    m = len(y)
    g = sigmoid(beta, X)
    return 1/m* np.sum(-y * np.log(g) - (1-y) * np.log(1-g))

# Objective function to minimize
def objective_function(beta, X, y, lambda_, gamma_):
    lloss=logistic_loss(beta, X, y, lambda_,gamma_)
    f=0 # not including any fairness constraint thus f=0
    l2loss=np.sum(beta**2)
    return lloss+gamma_*l2loss+lambda_*f

# Function for the given f prime
def fprime(beta, X, y, lambda_, gamma_):
    m = len(y)
    g = sigmoid(beta, X)
    return 1/m * np.dot(X.T,(g-y))+2*gamma_*beta

# The given evaluation error function
def evaluate_error(y_pred,labels_, group_):
    # amount of groups
    unique_groups = np.unique(group_)

    group_accuracy={g: np.nan for g in group_}

    for g in unique_groups:
        idx=group_==g
        group_accuracym=accuracy_score(y_pred[idx], labels_[idx])
        group_accuracy[g]=group_accuracym
    
    group_accuracy_df = pd.Series({int(k): float(v) for k, v in group_accuracy.items()}, name="Group Accuracy")
    
    overallaccuracy = accuracy_score(y_pred, labels_)
    
    return y_pred, overallaccuracy, group_accuracy_df

# Initial values
beta0 = np.zeros(X_train.shape[1])
lambda1=1
gamma1=1e-5


optimal_beta, nfeval, rc = fmin_tnc(func=objective_function,x0=beta0,fprime=fprime, args=(X_train,y_train, lambda1, gamma1), ftol=1e-5)
#print("Optimized beta:", optimal_beta)


y_pred_fair = np.array([True if x >=0.5 else False for x in sigmoid(optimal_beta, X_test)])



predictions, accuracy, group_acc = evaluate_error(y_pred_fair, y_test, groupsex_test)

# Group labels
group_labels = {1: "Male", 2: "Female"}

# Convert numeric keys to labels
group_acc = {group_labels.get(k, f"Unknown {k}"): v for k, v in group_acc.items()}

# Convert to pandas Series
group_acc_df = pd.Series(group_acc, name="Group Accuracy")


print("the accuracy is:", accuracy)
print("the group accuracy is:\n", group_acc_df)

the accuracy is: 0.7697339841054864
the group accuracy is:
 Male      0.786577
Female    0.750972
Name: Group Accuracy, dtype: float64


In [24]:
betas={'features': X_test.columns.tolist(), 'weight':optimal_beta}
df = pd.DataFrame(data=betas)
print("Estimated betas:\n" ,df)

Estimated betas:
      features    weight
0   Intercept -1.867255
1        AGEP  0.037373
2     HINS1_2 -1.026680
3     HINS2_2 -0.130875
4     HINS4_2  0.817298
5       CIT_2 -0.115007
6       CIT_3  0.006931
7       CIT_4  0.066099
8       CIT_5 -0.149606
9     COW_2.0 -0.035229
10    COW_3.0  0.049525
11    COW_4.0  0.070401
12    COW_5.0  0.609968
13    COW_6.0 -0.620537
14    COW_7.0  0.265462
15    COW_8.0 -1.186876
16   SCHL_2.0  0.340789
17   SCHL_3.0  0.418239
18   SCHL_4.0 -0.454315
19   SCHL_5.0 -0.184163
20   SCHL_6.0 -0.088570
21   SCHL_7.0  0.029908
22   SCHL_8.0 -0.192398
23   SCHL_9.0  0.007368
24  SCHL_10.0  0.124342
25  SCHL_11.0  0.061165
26  SCHL_12.0  0.196001
27  SCHL_13.0  0.073345
28  SCHL_14.0 -0.151037
29  SCHL_15.0  0.155557
30  SCHL_16.0  0.391373
31  SCHL_17.0  0.559916
32  SCHL_18.0  0.604334
33  SCHL_19.0  0.652756
34  SCHL_20.0  0.938767
35  SCHL_21.0  1.686284
36  SCHL_22.0  2.184330
37  SCHL_23.0  2.363544
38  SCHL_24.0  2.385018
39      MAR_2 -0.20079

## Race

In [25]:
optimal_beta, nfeval, rc = fmin_tnc(func=objective_function,x0=beta0,fprime=fprime, args=(X_train,y_train, lambda1, gamma1), ftol=1e-5)
print("Optimized beta:", optimal_beta)
y_pred_fair = np.array([True if x >=0.5 else False for x in sigmoid(optimal_beta, X_test)])


predictions, accuracy, group_acc = evaluate_error(y_pred_fair, y_test, grouprace_test)

# Group labels
group_labels = {1: "White alone", 2: "Black or African American alone", 3: "American Indian alone", 4: "Alaska Native alone", 5: "AI and AN tribes or AI or AN, not specified and no other races", 
                6: "Asian alone", 7: "Native Hawaiian and Other Pacific Islander alone", 8: "Some Other Race alone", 9: "Two or More Races"}


# Convert numeric keys to labels
group_acc = {group_labels.get(k, f"Unknown {k}"): v for k, v in group_acc.items()}

# Convert to pandas Series
group_acc_df = pd.Series(group_acc, name="Group Accuracy")


print("the accuracy is:", accuracy)
print("the group accuracy is:\n", group_acc_df)

Optimized beta: [-1.86725509  0.03737348 -1.02668025 -0.13087518  0.81729812 -0.11500725
  0.00693095  0.06609948 -0.14960641 -0.03522904  0.04952479  0.07040123
  0.60996828 -0.62053714  0.26546183 -1.18687588  0.34078904  0.41823864
 -0.45431505 -0.18416346 -0.08857031  0.02990758 -0.19239818  0.00736802
  0.12434218  0.06116496  0.19600091  0.07334509 -0.15103684  0.15555654
  0.39137259  0.55991573  0.60433434  0.65275592  0.93876708  1.68628434
  2.18432951  2.36354373  2.38501819 -0.20079884  0.03238696 -0.21520986
 -0.71368927 -0.81851541 -0.21723507 -0.17124925 -0.02560363 -0.20199104
 -0.03571581 -0.05024647 -0.05263261 -0.10452716 -0.29337948 -0.65141136
 -0.98235791  0.08991412]
the accuracy is: 0.7697339841054864
the group accuracy is:
 White alone                                                       0.774683
Some Other Race alone                                             0.751370
Black or African American alone                                   0.730860
Asian alone     

In [8]:
betas={'features': X_test.columns.tolist(), 'weight':optimal_beta}
df = pd.DataFrame(data=betas)
print("Estimated betas:\n" ,df)

Estimated betas:
      features    weight
0   Intercept -1.867255
1        AGEP  0.037373
2     HINS1_2 -1.026680
3     HINS2_2 -0.130875
4     HINS4_2  0.817298
5       CIT_2 -0.115007
6       CIT_3  0.006931
7       CIT_4  0.066099
8       CIT_5 -0.149606
9     COW_2.0 -0.035229
10    COW_3.0  0.049525
11    COW_4.0  0.070401
12    COW_5.0  0.609968
13    COW_6.0 -0.620537
14    COW_7.0  0.265462
15    COW_8.0 -1.186876
16   SCHL_2.0  0.340789
17   SCHL_3.0  0.418239
18   SCHL_4.0 -0.454315
19   SCHL_5.0 -0.184163
20   SCHL_6.0 -0.088570
21   SCHL_7.0  0.029908
22   SCHL_8.0 -0.192398
23   SCHL_9.0  0.007368
24  SCHL_10.0  0.124342
25  SCHL_11.0  0.061165
26  SCHL_12.0  0.196001
27  SCHL_13.0  0.073345
28  SCHL_14.0 -0.151037
29  SCHL_15.0  0.155557
30  SCHL_16.0  0.391373
31  SCHL_17.0  0.559916
32  SCHL_18.0  0.604334
33  SCHL_19.0  0.652756
34  SCHL_20.0  0.938767
35  SCHL_21.0  1.686284
36  SCHL_22.0  2.184330
37  SCHL_23.0  2.363544
38  SCHL_24.0  2.385018
39      MAR_2 -0.20079

## Logistic regression model SEX fairness 

In [26]:
# Distance function discrete
def d(y_i, y_j):
    return 1 if y_i == y_j else 0

# Group fairness function
def individual_fairness(beta, X, y, group):
    n1=np.sum(group==1)
    n2=np.sum(group==2)
    # The predictions
    pred1 = X[group==1] @ beta
    pred2 = X[group==2] @ beta
    # The target
    y1 = y[group==1]
    y2 = y[group==2]

    # Allocate space
    fairness_loss=0
    for i in range(n1):
        for j in range(n2):
            d_ij = d(y1[i], y2[j]) # distance
            fairness_loss += d_ij * (pred1[i] - pred2[j]) ** 2
    fairness_loss /= n1*n2
    return fairness_loss

def group_fairness(beta, X, y, group):
    n1=np.sum(group==1)
    n2=np.sum(group==2)
    # The predictions
    pred1 = X[group==1] @ beta
    pred2 = X[group==2] @ beta
    # The target
    y1 = y[group==1]
    y2 = y[group==2]

    # Allocate space
    fairness_loss=0
    for i in range(n1):
        for j in range(n2):
            d_ij = d(y1[i], y2[j]) # distance
            fairness_loss += d_ij * (pred1[i] - pred2[j]) 
    fairness_loss /= n1*n2
    return (fairness_loss)**2

# Sigmoid function
def sigmoid(beta, X):
    return 1/(1+np.exp(-(X @ beta)))

# Logistic loss
def logistic_loss(beta, X, y, lambda_, gamma_):
    m = len(y)
    g = sigmoid(beta, X)
    return 1/m* np.sum(-y * np.log(g) - (1-y) * np.log(1-g))

# Objective function
def objective_function(beta, X, y, lambda_, gamma_, group):
    lloss=logistic_loss(beta, X, y, lambda_,gamma_)
    f=group_fairness(beta, X, y, group) # can be changed with individual fairness
    l2loss=gamma_*np.sum(beta**2)
    return lloss+lambda_*f+l2loss

# Given f prime
def fprime(beta, X, y, lambda_, gamma_, group):
    m = len(y)
    g = sigmoid(beta, X)
    return 1/m * np.dot(X.T,(g-y))+2*gamma_*beta


In [29]:
## Testing for the ACS dataset
# Initial values
beta0 = np.zeros(X_train.shape[1])
lambda1=100
gamma1=1e-5


# I am subsampling because of runtime
N = 10000 ##
X_train = X_train[:N]
y_train = y_train[:N]
groupsex_train = groupsex_train [:N]


# Initial values
beta0 = np.zeros(X_train.shape[1])
lambda1=1
gamma1=1e-5


optimal_beta, nfeval, rc = fmin_tnc(func=objective_function,x0=beta0,fprime=fprime, args=(X_train,y_train, lambda1, gamma1, groupsex_train), ftol=1e-5)
#print("Optimized beta:", optimal_beta)


y_pred_fair = np.array([True if x >=0.5 else False for x in sigmoid(optimal_beta, X_test)])



predictions, accuracy, group_acc = evaluate_error(y_pred_fair, y_test, groupsex_test)

# Group labels
group_labels = {1: "Male", 2: "Female"}

# Convert numeric keys to labels
group_acc = {group_labels.get(k, f"Unknown {k}"): v for k, v in group_acc.items()}

# Convert to pandas Series
group_acc_df = pd.Series(group_acc, name="Group Accuracy")


print("the accuracy is:", accuracy)
print("the group accuracy is:\n", group_acc_df)


the accuracy is: 0.7534050545575346
the group accuracy is:
 Male      0.773095
Female    0.731471
Name: Group Accuracy, dtype: float64


In [11]:

predictions, accuracy, group_acc = evaluate_error(prediction_scoretest, y_test, grouprace_test)

# Group labels
group_labels = {1: "White alone", 2: "Black or African American alone", 3: "American Indian alone", 4: "Alaska Native alone", 5: "AI and AN tribes or AI or AN, not specified and no other races", 
                6: "Asian alone", 7: "Native Hawaiian and Other Pacific Islander alone", 8: "Some Other Race alone", 9: "Two or More Races"}


# Convert numeric keys to labels
group_acc = {group_labels.get(k, f"Unknown {k}"): v for k, v in group_acc.items()}

# Convert to pandas Series
group_acc_df = pd.Series(group_acc, name="Group Accuracy")


print("the accuracy is:", accuracy)
print("the group accuracy is:\n", group_acc_df)

the accuracy is: 0.7697339841054864
the group accuracy is:
 White alone                                                       0.774683
Some Other Race alone                                             0.751370
Black or African American alone                                   0.730860
Asian alone                                                       0.776458
Two or More Races                                                 0.766344
American Indian alone                                             0.776000
AI and AN tribes or AI or AN, not specified and no other races    0.811111
Native Hawaiian and Other Pacific Islander alone                  0.663717
Alaska Native alone                                               0.400000
Name: Group Accuracy, dtype: float64


## Logistic regression model RAC1P fairness

In [12]:
print("test")

test
